In [1]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from PIL import Image
from tqdm import tqdm

# 确保 PIL 版本兼容 (用于 resize)
try:
    # PIL 9.0.0+
    LANCZOS_RESAMPLE = Image.Resampling.LANCZOS
except AttributeError:
    # Older PIL
    LANCZOS_RESAMPLE = Image.LANCZOS

# --- 1. 定义常量和路径 ---
# 请根据您的环境修改这里的根路径
DATA_ROOT = "/mnt/hdd/jiazy/petfinder-pawpularity-score"
METADATA_FILE = os.path.join(DATA_ROOT, "train.csv")
IMAGE_DIR = os.path.join(DATA_ROOT, "train")

# 输出目录
OUTPUT_DIR = "./features"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# --- 2. 定义特征列 ---
# 连续特征
CONTINUOUS_COLS = []

# 类别特征
CATEGORICAL_COLS = [
    'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage'
]

# 标签列
LABEL_COL = 'Pawpularity'

# 非特征列 (将被移除)
NON_FEATURE_COLS = [
    'Id', 
]

# --- 3. 图像处理辅助函数 ---
def process_and_save_image(png_path):
    """
    加载 PNG 图像, resize 到 224x224, 并另存为 .npy 文件。
    如果 .npy 文件已存在，则跳过。
    """
    if not os.path.exists(png_path):
        print(f"警告：图像文件不存在 {png_path}")
        return None  # 返回 None 以便后续过滤

    # 将 .png 替换为 .npy
    npy_path = png_path.replace(".jpg", ".npy")

    if os.path.exists(npy_path):
        return npy_path  # 文件已存在

    try:
        # 打开图像
        img = Image.open(png_path)
        
        # Resize 到 224x224
        img_resized = img.resize((224, 224), resample=LANCZOS_RESAMPLE)
        
        # 转换为 NumPy 数组
        if img_resized.mode == 'RGBA':
            img_resized = img_resized.convert('RGB')
            
        np_img = np.array(img_resized)
        
        # 保存为 .npy
        np.save(npy_path, np_img)
        
        return npy_path
    except Exception as e:
        print(f"处理图像失败 {png_path}: {e}")
        return None

def preprocess_petfinder_flow(metadata_file, image_dir, output_dir, label_col):
    """
    预处理 PetFinder 数据集
    """
    print("开始执行 PetFinder 数据集预处理流程...")
    os.makedirs(output_dir, exist_ok=True)
    
    # --- 1. 📖 加载数据 ---
    print(f"正在加载数据 {metadata_file}...")
    df = pd.read_csv(metadata_file)
    print(f"成功加载 {len(df)} 条记录。")

    print("正在构造图像路径...")
    df['image_path'] = df['Id'].apply(lambda x: os.path.join(image_dir, f"{x}.jpg"))


    # --- 2. 📝 定义特征 ---
    print("正在定义特征...")
    continuous_cols = CONTINUOUS_COLS
    categorical_cols = CATEGORICAL_COLS
    non_feature_cols = NON_FEATURE_COLS
    exclude_cols = [label_col, 'image_path', 'npy_path'] + non_feature_cols
    
    # --- 3. 🧹 数据清洗 ---
    print("正在清洗数据...")
    
    # 过滤缺失图像路径和标签的行
    df = df.dropna(subset=[label_col])
    print("正在进行类别特征编码...")
    cat_dims = [] # 记录每个特征的类别数
    for col in categorical_cols:
        df[col] = df[col].fillna(-1) # 简单填充
        df[col] = df[col].astype('category')
        df[col] = df[col].cat.codes
        # 记录类别数量 (最大ID + 1)
        cat_dims.append(int(df[col].max() + 1))

    # --- 6. 数据集划分 ---
    print("正在进行 80:10:10 数据集划分...")
    # 回归任务通常不需要 stratify，或者需要分桶 stratify。简单起见这里用随机划分。
    train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
    
    print(f"划分结果: 训练集 {len(train_df)}, 验证集 {len(val_df)}, 测试集 {len(test_df)}")

    # --- 7. 连续特征标准化 (如果有) ---
    if continuous_cols:
        print("正在标准化连续特征...")
        scaler = StandardScaler()
        scaler.fit(train_df[continuous_cols])
        train_df[continuous_cols] = scaler.transform(train_df[continuous_cols])
        val_df[continuous_cols] = scaler.transform(val_df[continuous_cols])
        test_df[continuous_cols] = scaler.transform(test_df[continuous_cols])

    # --- 8. 图像处理 ---
    print("正在处理图像 (Resize & Save .npy)...")
    tqdm.pandas(desc="Train Images")
    train_df['npy_path'] = train_df['image_path'].progress_apply(process_and_save_image)
    
    tqdm.pandas(desc="Val Images")
    val_df['npy_path'] = val_df['image_path'].progress_apply(process_and_save_image)
    
    tqdm.pandas(desc="Test Images")
    test_df['npy_path'] = test_df['image_path'].progress_apply(process_and_save_image)

    # 过滤掉图像处理失败的 (返回 None 的)
    len_before = len(train_df) + len(val_df) + len(test_df)
    train_df = train_df.dropna(subset=['npy_path'])
    val_df = val_df.dropna(subset=['npy_path'])
    test_df = test_df.dropna(subset=['npy_path'])
    len_after = len(train_df) + len(val_df) + len(test_df)
    if len_before != len_after:
        print(f"警告：过滤了 {len_before - len_after} 条图像不存在或损坏的数据。")

    # --- 9. 保存输出 ---
    print(f"[Final] 正在保存处理后的文件到 {output_dir} ...")

    # 保存字段长度: 先类别，后连续 (符合 TIP 模型要求)
    tabular_lengths = cat_dims + [1] * len(continuous_cols)
    torch.save(tabular_lengths, os.path.join(output_dir, "tabular_lengths.pt"))
    
    for split_name, df_split in zip(["train", "val", "test"], [train_df, val_df, test_df]):
        features_path = os.path.join(output_dir, f"{split_name}_features.csv")
        # 确保列顺序：先类别，后连续
        cols_to_save = categorical_cols + continuous_cols
        df_split[cols_to_save].to_csv(features_path, index=False, header=False)
        
        labels_path = os.path.join(output_dir, f"{split_name}_labels.pt")
        # [修正] 标签转为 float32 用于回归
        labels_tensor = torch.tensor(df_split[label_col].values, dtype=torch.float32)
        torch.save(labels_tensor, labels_path)
        
        paths_path = os.path.join(output_dir, f"{split_name}_paths.pt")
        # 保存 .npy 的绝对路径
        npy_path_list = df_split['npy_path'].tolist()
        torch.save(npy_path_list, paths_path)

    print("预处理全部完成！")

if __name__ == "__main__":
    preprocess_petfinder_flow(
        metadata_file=METADATA_FILE,
        image_dir=IMAGE_DIR,
        output_dir=OUTPUT_DIR,
        label_col=LABEL_COL
    )

开始执行 PetFinder 数据集预处理流程...
正在加载数据 /mnt/hdd/jiazy/petfinder-pawpularity-score/train.csv...
成功加载 9912 条记录。
正在构造图像路径...
正在定义特征...
正在清洗数据...
正在进行类别特征编码...
正在进行 80:10:10 数据集划分...
划分结果: 训练集 7929, 验证集 991, 测试集 992
正在处理图像 (Resize & Save .npy)...


Test Images: 100%|██████████| 992/992 [00:31<00:00, 31.56it/s]


[Final] 正在保存处理后的文件到 ./features ...
预处理全部完成！
